In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import folium
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import pandas as pd
from sympy import *
import matplotlib.pyplot as plt
from folium import plugins

# DAEGU = [35.85, 128.56]
# BUSAN = [35.1795543, 129.0756416]
JEJU = [33.30, 126.54]
KADIZ = [(39.00, 123.30),
         (37.00, 124.00),
         (30.00, 124.00),
         (30.00, 124.00),
         (30.00, 125.25),
         (30.00, 125.25),
         (32.30, 126.50),
         (32.30, 127.30),
         (34.17, 128.52),
         (35.13, 129.48),
         (36.00, 130.30),
         (37.17, 133.00),
         (39.00, 133.00),
         (39.00, 123.30)]
IEODO = [32.11753829417425, 125.16666665729801]
DOKDO = [37.24315823449135, 131.86690646281266]

def init_map():
    m = folium.Map(location = JEJU, zoom_start = 6)
    folium.PolyLine(locations = KADIZ,tooltip = 'KADIZ', color='red').add_to(m)
    folium.Marker(IEODO, popup="<i>IEODO</i>").add_to(m)
    folium.Marker(DOKDO, popup="<i>DOKDO</i>").add_to(m)
    return m

m = init_map()

m

In [2]:
def mk_row(point):
    x, y = point
    return [y * y, x, y, x * y, 1]

#return value (a, b, c, d, e, f) matches (ax^2 + by^2 + cx + by + exy + f = 0)
def find_ellipse(points):
    if len(points) != 5:
        raise ValueError
    else:
        mat = []
        constant = []
        for point in points:
            mat.append(mk_row(point))
            constant.append(-point[0]**2)
        sol = np.linalg.solve(np.array(mat), constant)
    return np.concatenate((np.array([1]), sol), axis = None)

def make_route(sol, data, input_axis, select):
    x = symbols('x', real = True)
    y = symbols('y', real = True)
    a, b, c, d, e, f = sol
    route = []
    if input_axis == 'x':
        for i in data:
            x = i
            ans = solve(a*x**2 + b*y**2 + c*x + d*y + e*x*y + f)
            if len(ans) == 0:
                print('No real solution at {}'.format(i))
                break
            if select == 'max':
                y_sol = max(ans)
                route.append([y_sol, i])
            elif select == 'min':
                y_sol = min(ans)
                route.append([y_sol, i])
    elif input_axis == 'y':
        for i in data:
            y = i
            ans = solve(a*x**2 + b*y**2 + c*x + d*y + e*x*y + f)
            if len(ans) == 0:
                print('No real solution at {}'.format(i))
                break
            if select == 'max':
                x_sol = max(ans)
                route.append([i, x_sol])
            elif select == 'min':
                x_sol = min(ans)
                route.append([i, x_sol])
    return route

In [3]:
# Make base route
points = [(120.15, 29.331), (125.8, 31.369), (127.42, 32.459), (129.44, 34.436), (131.35, 38.448)]

sol = find_ellipse(points)

lat_range = np.linspace(123, 131, 100)
base_route = np.array(make_route(sol, lat_range, 'x', 'min'))

print(base_route.shape)

# Make turning route
points_turing = [(122.6, 30.749), (126.4, 28.964), (128.1, 27.674), (130.63, 24.392), (131.37, 17.255)]

sol_turning = find_ellipse(points_turing)
base_turning_route = np.array(make_route(sol_turning, lat_range, 'x', 'max'))
print(base_turning_route.shape)

(100, 2)
(100, 2)


In [4]:
folium.PolyLine(locations = base_route, tooltip = 'flight tracks', color = 'blue').add_to(m)
folium.PolyLine(locations = base_turning_route, tooltip = 'turning tracks', color = 'blue').add_to(m)
m

In [5]:
# make noise
mu, sigma = 0.1, 0.01
routes = []
route_turning = []
# route_dataset = [[] for _ in range(900)]

for i in range(100):
    s = np.random.default_rng(i).normal(mu, sigma, size=(100, 2))
    routes.append(base_route + s)
route_dataset = np.array(routes)

for i in range(100):
    s = np.random.default_rng(i).normal(mu, sigma, size=(100, 2))
    route_turning.append(base_turning_route + s)
route_turning_dataset = np.array(route_turning)

# window_size = 20
# offset = window_size // 2 - 1

# for i in range(len(routes)):
#     for j in range(0, len(routes[0]) - window_size + 1, window_size // 2):
#         pos = i * offset + j // offset
# #         print('i:%5d, j:%5d, j+w_size:%5d, pos:%5d' % (i, j, j+window_size, pos))
#         route_dataset[pos].append(routes[i:i+1, j:j+window_size, :])
    
# route_dataset = np.array(route_dataset)
# route_dataset = np.squeeze(route_dataset)
# print(route_dataset.shape)
# print(np.array_equal(routes[0, 0:window_size, :],
#                      route_dataset[0, 0:window_size, :]))
print(route_dataset.shape)
print(route_turning_dataset.shape)

(100, 100, 2)
(100, 100, 2)


In [6]:
# show one route in dataset 
# folium.PolyLine(locations = route_dataset[0:window_size//2], tooltip = 'track with noise', color = 'orange').add_to(m)
folium.PolyLine(locations = route_dataset[0], tooltip = 'track with noise', color = 'orange').add_to(m)
folium.PolyLine(locations = route_turning_dataset[0], tooltip = 'track with noise', color = 'orange').add_to(m)
m

In [7]:
# Split datasets
ratio = 0.7
boundary = int(len(route_dataset) * ratio)

train_dataset = route_dataset[:boundary, :, :]
test_dataset = route_dataset[boundary:, :, :]

print(train_dataset.shape, test_dataset.shape)

(70, 100, 2) (30, 100, 2)


In [8]:
# Split turning datasets
ratio = 0.7
boundary = int(len(route_turning_dataset) * ratio)

train_turning_dataset = route_turning_dataset[:boundary, :, :]
test_turning_dataset = route_turning_dataset[boundary:, :, :]

print(train_turning_dataset.shape, test_turning_dataset.shape)

(70, 100, 2) (30, 100, 2)


In [9]:
standard_base = StandardScaler()
minmax_base = MinMaxScaler()
standard_turning = StandardScaler()
minmax_turning = MinMaxScaler()

class RouteDataset(Dataset): 
    def __init__(self, train=True, base=True):    
        # Scaling
        if (base):
            self.dataset = train_dataset if train == True else test_dataset
        else:
            self.dataset = train_turning_dataset if train == True else test_turning_dataset
        self.boundary = int(len(self.dataset[0]) * 0.05)
        
        self.x_data = self.dataset[:, :self.boundary, :]
        self.y_data = self.dataset[:, self.boundary:, :]
        
        if(base):
            self.x_data = standard_base.fit_transform(self.x_data.reshape(-1, self.x_data.shape[-1])).reshape(self.x_data.shape)
            self.y_data = minmax_base.fit_transform(self.y_data.reshape(-1, self.y_data.shape[-1])).reshape(self.y_data.shape)
        else:
            self.x_data = standard_turning.fit_transform(self.x_data.reshape(-1, self.x_data.shape[-1])).reshape(self.x_data.shape)
            self.y_data = minmax_turning.fit_transform(self.y_data.reshape(-1, self.y_data.shape[-1])).reshape(self.y_data.shape) 
            
    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [11]:
batch_size = 16

train_loader = DataLoader(dataset=RouteDataset(train=True), 
                          batch_size=batch_size)
test_loader = DataLoader(dataset=RouteDataset(train=False), 
                         batch_size=batch_size)

train_turning_loader = DataLoader(dataset=RouteDataset(train=True, base=False), 
                                  batch_size=batch_size)
test_turning_loader = DataLoader(dataset=RouteDataset(train=False, base=False), 
                                 batch_size=batch_size)

for i, (x, y) in enumerate(train_loader):
    print(i, x.shape, y.shape)
for i, (x, y) in enumerate(test_loader):
    print(i, x.shape, y.shape)
for i, (x, y) in enumerate(train_turning_loader):
    print(i, x.shape, y.shape)
for i, (x, y) in enumerate(test_turning_loader):
    print(i, x.shape, y.shape)

0 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
1 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
2 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
3 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
4 torch.Size([6, 5, 2]) torch.Size([6, 95, 2])
0 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
1 torch.Size([14, 5, 2]) torch.Size([14, 95, 2])
0 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
1 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
2 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
3 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
4 torch.Size([6, 5, 2]) torch.Size([6, 95, 2])
0 torch.Size([16, 5, 2]) torch.Size([16, 95, 2])
1 torch.Size([14, 5, 2]) torch.Size([14, 95, 2])


In [12]:
# Model definition
device = 'cpu'

class LSTM(nn.Module):
    def __init__(self,output_size,input_size,hidden_size,num_layers,seq_length):
        super(LSTM,self).__init__()
        self.output_size = output_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size , hidden_size=hidden_size , num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size,128)
        self.fc = nn.Linear(128,output_size)

        self.relu = nn.ReLU()

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0),self.hidden_size)).to(device)

        output, (hn,cn) = self.lstm(x,(h_0,c_0))

        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)
        return out

In [13]:
num_epochs = 100
learning_rate = 0.0001

input_size = 2
hidden_size = 32
num_layers = 1
# output_size = 100
# seq_length = 50
# output_size = 20
# seq_length = 10
total_seq_length = 100
seq_length = 5
output_size = (total_seq_length - seq_length) * input_size

print_every = 10

base_model = LSTM(output_size, input_size, hidden_size, num_layers, seq_length)
turning_model = LSTM(output_size, input_size, hidden_size, num_layers, seq_length)

loss_function = torch.nn.MSELoss()
base_optimizer = torch.optim.Adam(base_model.parameters(), lr = learning_rate)
turning_optimizer = torch.optim.Adam(turning_model.parameters(), lr = learning_rate)

In [14]:
# Base Model trainig
for epoch in range(1, num_epochs+1):
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = Variable(x_train.view(-1, seq_length, input_size))
        y_train = Variable(y_train.view(-1, output_size))

        base_optimizer.zero_grad()
        outputs = base_model(x_train)
#         import pdb; pdb.set_trace()
        loss = loss_function(outputs, y_train)

        loss.backward()
        base_optimizer.step()

    if epoch % print_every == 0:
        print("[Epoch %3d] loss : %1.5f" % (epoch, loss.item()))

[Epoch  10] loss : 0.24536
[Epoch  20] loss : 0.21116
[Epoch  30] loss : 0.16562
[Epoch  40] loss : 0.10500
[Epoch  50] loss : 0.04262
[Epoch  60] loss : 0.00931
[Epoch  70] loss : 0.00203
[Epoch  80] loss : 0.00055
[Epoch  90] loss : 0.00016
[Epoch 100] loss : 0.00007


In [15]:
# Turning Model trainig
for epoch in range(1, num_epochs+1):
    for i, (x_train, y_train) in enumerate(train_turning_loader):
        x_train = Variable(x_train.view(-1, seq_length, input_size))
        y_train = Variable(y_train.view(-1, output_size))

        turning_optimizer.zero_grad()
        outputs = turning_model(x_train)
#         import pdb; pdb.set_trace()
        loss = loss_function(outputs, y_train)

        loss.backward()
        turning_optimizer.step()

    if epoch % print_every == 0:
        print("[Epoch %3d] loss : %1.5f" % (epoch, loss.item()))

[Epoch  10] loss : 0.38913
[Epoch  20] loss : 0.34506
[Epoch  30] loss : 0.27998
[Epoch  40] loss : 0.18227
[Epoch  50] loss : 0.06669
[Epoch  60] loss : 0.01185
[Epoch  70] loss : 0.00212
[Epoch  80] loss : 0.00040
[Epoch  90] loss : 0.00010
[Epoch 100] loss : 0.00004


In [16]:
# Base Model testing
total, total_loss = 0, 0.0
base_model.eval()

for i, (x_test, y_test) in enumerate(test_loader):
    x_test = Variable(x_test.view(-1, seq_length, input_size))
    y_test = Variable(y_test.view(-1, output_size))

    outputs = base_model(x_test)
    loss = loss_function(outputs, y_test)

    total += y_test.size(0)
    total_loss += loss.item()
        
avg_loss = total_loss / total
print('Test Loss : %1.5f' % avg_loss)

Test Loss : 0.00000


In [17]:
# Turning Model testing
total, total_loss = 0, 0.0
turning_model.eval()

for i, (x_test, y_test) in enumerate(test_turning_loader):
    x_test = Variable(x_test.view(-1, seq_length, input_size))
    y_test = Variable(y_test.view(-1, output_size))

    outputs = turning_model(x_test)
    loss = loss_function(outputs, y_test)

    total += y_test.size(0)
    total_loss += loss.item()
        
avg_loss = total_loss / total
print('Test Loss : %1.5f' % avg_loss)

Test Loss : 0.00001


In [18]:
for (x, y) in test_loader:
  print(x.size())
  print(x)

torch.Size([16, 5, 2])
tensor([[[-1.3634, -1.4988],
         [-0.4771, -0.7431],
         [ 0.3066, -0.1261],
         [ 0.7701,  0.7063],
         [ 1.5195,  1.3414]],

        [[-1.3331, -1.3651],
         [-0.7104, -0.7120],
         [ 0.0996,  0.0465],
         [ 0.7431,  0.5839],
         [ 1.2230,  1.3874]],

        [[-1.5747, -1.4128],
         [-0.5507, -0.8517],
         [-0.3365, -0.0818],
         [ 0.8941,  0.8053],
         [ 1.4274,  1.4533]],

        [[-1.5953, -1.3424],
         [-0.7727, -0.6784],
         [ 0.0615, -0.1682],
         [ 0.7790,  0.6236],
         [ 1.0682,  1.5117]],

        [[-1.2877, -1.3923],
         [-0.5160, -0.7242],
         [-0.2142, -0.0262],
         [ 0.6735,  0.7234],
         [ 1.3089,  1.4853]],

        [[-1.5988, -1.3757],
         [-0.5919, -0.6661],
         [ 0.1013,  0.0041],
         [ 0.7661,  0.8009],
         [ 1.7785,  1.4009]],

        [[-1.4753, -1.2574],
         [-0.4601, -0.6599],
         [ 0.2306, -0.1348],
        

In [19]:
# Visualization (Base)
for (x, y) in test_loader:
    x = Variable(x[0].view(-1, seq_length, input_size))
    y = Variable(y[0].view(-1, output_size))
    y_predicted = base_model(x)
    
    x = x.view(seq_length, input_size).data.detach().cpu().numpy()
    y = y.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()
    y_predicted = y_predicted.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()    
        
    x = standard_base.inverse_transform(x)
    y = minmax_base.inverse_transform(y)
    y_predicted = minmax_base.inverse_transform(y_predicted)
    
    m = init_map()
    
    folium.PolyLine(locations = x,color = 'blue',tooltip = 'Real Trajectory').add_to(m)
    folium.PolyLine(locations = y,color = 'green',tooltip = 'Predicted Trajectory').add_to(m)
    folium.PolyLine(locations = y_predicted,color = 'orange',tooltip = 'Predicted Trajectory').add_to(m)
    
    break
m

In [20]:
# Visualization (Turning)
for (x, y) in test_turning_loader:
    x = Variable(x[0].view(-1, seq_length, input_size))
    y = Variable(y[0].view(-1, output_size))
    y_predicted = turning_model(x)
    
    x = x.view(seq_length, input_size).data.detach().cpu().numpy()
    y = y.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()
    y_predicted = y_predicted.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()    
        
    x = standard_turning.inverse_transform(x)
    y = minmax_turning.inverse_transform(y)
    y_predicted = minmax_turning.inverse_transform(y_predicted)
    
    m = init_map()
    
    folium.PolyLine(locations = x,color = 'blue',tooltip = 'Real Trajectory').add_to(m)
    folium.PolyLine(locations = y,color = 'green',tooltip = 'Predicted Trajectory').add_to(m)
    folium.PolyLine(locations = y_predicted,color = 'orange',tooltip = 'Predicted Trajectory').add_to(m)
    
    break
m

In [21]:
# Animiation

lines = []

for (x, y) in test_loader:
    x = Variable(x[0].view(-1, seq_length, input_size))
    y = Variable(y[0].view(-1, output_size))
    y_predicted = base_model(x)
    
    x = x.view(seq_length, input_size).data.detach().cpu().numpy()
    y = y.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()
    y_predicted = y_predicted.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()    
        
    x = standard_base.inverse_transform(x)
    y = minmax_base.inverse_transform(y)
    y_predicted = minmax_base.inverse_transform(y_predicted)
    
    # Swap latitude, longitude 
    x[:,[0, 1]] = x[:,[1, 0]]
    y[:,[0, 1]] = y[:,[1, 0]]
    y_predicted[:,[0, 1]] = y_predicted[:,[1, 0]]
     
    lines.append({
        "coordinates": x.tolist(),
        "dates": ['2022-09-05T{hour:02d}:{minute:02d}:00'.format(hour=0, minute=5 * i) for i in range(len(x))],
        "color": "blue"
    })
    lines.append({
        "coordinates": y_predicted.tolist(),
        "dates": ['2022-09-05T{hour:02d}:{minute:02d}:00'.format(hour=1 + i // 60, 
                                                                 minute=i % 60) for i in range(len(y_predicted))],
        "color": "orange"
    })
    lines.append({
        "coordinates": y.tolist(),
        "dates": ['2022-09-05T{hour:02d}:{minute:02d}:00'.format(hour=3 + i // 60, 
                                                                 minute=i % 60) for i in range(len(y))],
        "color": "blue"
    })
    
    break

for (x, y) in test_turning_loader:
    x = Variable(x[0].view(-1, seq_length, input_size))
    y = Variable(y[0].view(-1, output_size))
    y_predicted = turning_model(x)
    
    x = x.view(seq_length, input_size).data.detach().cpu().numpy()
    y = y.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()
    y_predicted = y_predicted.view((total_seq_length - seq_length), input_size).data.detach().cpu().numpy()    
        
    x = standard_turning.inverse_transform(x)
    y = minmax_turning.inverse_transform(y)
    y_predicted = minmax_turning.inverse_transform(y_predicted)
    
    # Swap latitude, longitude 
    x[:,[0, 1]] = x[:,[1, 0]]
    y[:,[0, 1]] = y[:,[1, 0]]
    y_predicted[:,[0, 1]] = y_predicted[:,[1, 0]]

    y_predicted = y_predicted[2:, :]
     
    lines.append({
        "coordinates": y_predicted.tolist(),
        "dates": ['2022-09-05T{hour:02d}:{minute:02d}:00'.format(hour=1 + i // 60, 
                                                                 minute=i % 60) for i in range(len(y_predicted))],
        "color": "orange"
    })
    
    break

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"]
            },
        },
    }
    for line in lines
]

m = init_map()
plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    transition_time=100,
    add_last_point=False,
).add_to(m)

header_html = '''
             <div style='display:flex; justify-content:space-around;'>
             <h3 style="font-size:16px; color:blue; text-align:left;">파란색 : 실제 항적경로</h3>
             <h3 style="font-size:16px; color:orange; text-align:left;">주황색 : LSTM 모델의 예측 항적경로</h3>
             </div>
             '''

m.get_root().html.add_child(folium.Element(header_html))

m